# Projeto 1 - Ciência dos Dados

Nome: _____

Nome: _____

Nome: _____

___
### Atenção: <br> $\quad$* Serão permitidos grupos de até 3 pessoas, mas com uma rubrica mais exigente. <br> $\quad$* Veja RUBRICA na última página do arquivo PDF que traz o enunciado do Proejto 1.

___
Carregando algumas bibliotecas:

In [3]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

import re 


def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    #import string
    punctuation = '[´"!-.:?;$''<>]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    return text_subbed


In [4]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
c:\Users\euleo\Documents\GitHub\Projeto-1---CDados


___

### Carregando a base de dados com as mensagens dos seus arquivos. <br> Tire o `#` do início da linha de código condizente ao caso escolhido para o projeto 1.

In [10]:
nome = "danielr"

In [11]:
# SE SEU PROJETO USA OS DADOS ATÉ TRIO
#train = pd.read_csv('dados_treino_ate_TRIO_'+nome+'.csv')
#test = pd.read_csv('dados_teste_ate_TRIO_'+nome+'.csv')

In [5]:
# SE SEU PROJETO USA OS DADOS SÓ DUPLA"
train = pd.read_csv('dados_treino_so_DUPLA_danielr.csv')
test = pd.read_csv('dados_teste_so_DUPLA_danielr.csv')

In [6]:
train.head()

,Text,target
0,Fi-shock works great. This is number three fo...,1.0
1,Great stuff delivery is very fast . I mix it w...,1.0
2,I purchased these for my mom and she loves the...,1.0
3,I love Eight O'clock coffee but when shopping ...,0.0
4,These are really good. You do need to add jus...,1.0


In [7]:
test.head()

,Text,target
0,I ordered a case of 10 for $[...].00. I love t...,1.0
1,I bought these based on all the great reviews ...,0.0
2,This is a fantastic product. I promote this pr...,0.0
3,A nicely spiced light roast coffee with the ar...,1.0
4,This really is a DELICIOUS product and if you ...,0.0


___
# Classificador Automático (Boot)


Faça aqui uma descrição do seu assunto e o contexto referente aos rótulos cujas mensagens (ou reviews) deverão ser classificadas.

ESCREVA AQUI...

criar uma base de dados filtrando todos os comentarios com target 1 e outra com target 0


___
## Montando SEU Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [ ]:
# filtrar por linhas que o valor da coluna target seja igual a 1
um = train.loc[train.target == 1,:]
zero = train.loc[train.target == 0,:]

um =  um.Text
zero = zero.Text

umstr = ' '.join(um.astype(str).tolist())
zerostr = ' '.join(zero.astype(str).tolist())

umlimpo = cleanup(umstr.lower())
zerolimpo = cleanup(zerostr.lower())

umlista = umlimpo.split()
zerolista = zerolimpo.split()
trainlista = umlista + zerolista

umseries = pd.Series(umlista)
zeroseries = pd.Series(zerolista)
trainseries = pd.Series(trainlista)

# contagens absolutas (necessárias para smoothing de Laplace)
umcount    = umseries.value_counts()
zerocount  = zeroseries.value_counts()
traincount = trainseries.value_counts()

# parâmetros p/ Laplace
alpha = 1.0
V = traincount.index.nunique()             # tamanho do vocabulário

# totais de tokens por "classe" e no geral
total_um_tokens    = int(umcount.sum())
total_zero_tokens  = int(zerocount.sum())
total_train_tokens = int(traincount.sum())

# penalidades (probabilidade de OOV) suavizadas por Laplace
penalidade_um    = alpha / (total_um_tokens    + alpha * V)
penalidade_zero  = alpha / (total_zero_tokens  + alpha * V)
penalidade_train = alpha / (total_train_tokens + alpha * V)


In [ ]:
lenum = len(um)
lenzero = len(zero)

probum = lenum/(lenum + lenzero)
probzero = lenzero/(lenum + lenzero)

penalidade = 0.000005

def classcome(come):
    come = come.lower()
    # comestr = ''.join(come.astype(str).tolist())
    comelimpo = cleanup(come)
    comelista = comelimpo.split()

    probcomedadoum = 1
    probcomedadozero = 1
    
    probcome = 1

    for palavra in comelista:
        # P(palavra | classe=1) com Laplace
        c_um = umcount.get(palavra, 0)
        p_um = (c_um + alpha) / (total_um_tokens + alpha * V)
        probcomedadoum *= p_um

        # P(palavra | classe=0) com Laplace
        c_zero = zerocount.get(palavra, 0)
        p_zero = (c_zero + alpha) / (total_zero_tokens + alpha * V)
        probcomedadozero *= p_zero

        # P(palavra) no corpus todo (para manter sua razão com "probcome")
        c_train = traincount.get(palavra, 0)
        p_train = (c_train + alpha) / (total_train_tokens + alpha * V)
        probcome *= p_train

        # mantém seus fatores (não são necessários em log, mas você pediu sem mudar a lógica)
        probcomedadoum   = probcomedadoum * 1000
        probcomedadozero = probcomedadozero * 1000
        probcome         = probcome * 1000

        # print(probcomedadoum)
        # print(probcomedadozero)
        # print(probcome)
    
    # print(probcomedadoum)
    # print(probcomedadozero)
    # print(probcome)
        
    probumdadocome = probcomedadoum * probum / probcome
    probzerodadocome= probcomedadozero * probzero / probcome

    # print(probumdadocome)
    # print(probzerodadocome)
        
    if probumdadocome > probzerodadocome:
        return 1
    else:
        return 0

___
## Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [ ]:
avaliado = train

TP = TN = FP = FN = 0
total = len(avaliado)

for i in range(total):
    pred = classcome(avaliado.iloc[i, 0])     # coluna Text
    real = int(avaliado.iloc[i, 1])           # coluna target (0/1)

    if pred == 1 and real == 1:
        TP += 1
    elif pred == 0 and real == 0:
        TN += 1
    elif pred == 1 and real == 0:
        FP += 1
    elif pred == 0 and real == 1:
        FN += 1

acertos = TP + TN
acuracia = acertos / total if total else 0.0

# porcentagens sobre o total
pct_TP = (TP / total) * 100 if total else 0.0
pct_TN = (TN / total) * 100 if total else 0.0
pct_FP = (FP / total) * 100 if total else 0.0
pct_FN = (FN / total) * 100 if total else 0.0

print(f"Acuracia: {acuracia:.2%} ({acertos}/{total})")
print(f"Verdadeiro Positivo (TP): {pct_TP:.2f}%  ({TP})")
print(f"Verdadeiro Negativo (TN): {pct_TN:.2f}%  ({TN})")
print(f"Falso  Positivo  (FP): {pct_FP:.2f}%  ({FP})")
print(f"Falso  Negativo  (FN): {pct_FN:.2f}%  ({FN})")

Acurácia: 93.75% (1575/1680)
Falsos positivos: 4.82% (81/1680)
Falsos negativos: 1.43% (24/1680)


In [ ]:
avaliado = test

TP = TN = FP = FN = 0
total = len(avaliado)

for i in range(total):
    pred = classcome(avaliado.iloc[i, 0])     # coluna Text
    real = int(avaliado.iloc[i, 1])           # coluna target (0/1)

    if pred == 1 and real == 1:
        TP += 1
    elif pred == 0 and real == 0:
        TN += 1
    elif pred == 1 and real == 0:
        FP += 1
    elif pred == 0 and real == 1:
        FN += 1

acertos = TP + TN
acuracia = acertos / total if total else 0.0

# porcentagens sobre o total
pct_TP = (TP / total) * 100 if total else 0.0
pct_TN = (TN / total) * 100 if total else 0.0
pct_FP = (FP / total) * 100 if total else 0.0
pct_FN = (FN / total) * 100 if total else 0.0

print(f"Acuracia: {acuracia:.2%} ({acertos}/{total})")
print(f"Verdadeiro Positivo (TP): {pct_TP:.2f}%  ({TP})")
print(f"Verdadeiro Negativo (TN): {pct_TN:.2f}%  ({TN})")
print(f"Falso  Positivo  (FP): {pct_FP:.2f}%  ({FP})")
print(f"Falso  Negativo  (FN): {pct_FN:.2f}%  ({FN})")

C:\Users\euleo\AppData\Local\Temp\ipykernel_28300\4038290373.py:47: RuntimeWarning: invalid value encountered in scalar divide
  probumdadocome = probcomedadoum * probum / probcome
C:\Users\euleo\AppData\Local\Temp\ipykernel_28300\4038290373.py:48: RuntimeWarning: invalid value encountered in scalar divide
  probzerodadocome= probcomedadozero * probzero / probcome


Acurácia: 72.92% (525/720)
Falsos positivos: 15.69% (113/720)
Falsos negativos: 11.39% (82/720)


___
## Análise Qualitativa da Performance do Classificador

In [ ]:
# Analisando a acuracia obtida no treino de 93.75% e no teste de 72.92%, é possivel afirmar que embora ele não seja
# absurdamente preciso, a sua precisão é bem alta para o tipo de analise que é buscado, pois os criterios não são totalmente
# objetivos, com comentarios podendo variar dependendo do que o usuario pensa além de que existe a possibilidade de erros humanos
# na hora de realizar uma avaliação 


C:\Users\euleo\AppData\Local\Temp\ipykernel_28300\4038290373.py:47: RuntimeWarning: invalid value encountered in scalar divide
  probumdadocome = probcomedadoum * probum / probcome
C:\Users\euleo\AppData\Local\Temp\ipykernel_28300\4038290373.py:48: RuntimeWarning: invalid value encountered in scalar divide
  probzerodadocome= probcomedadozero * probzero / probcome


Acuracia: 72.92% (525/720)


___
## Qualidade do Classificador a partir de novas separações das mensagens entre Treinamento e Teste

Caso for fazer esse item do Projeto

___
## Aperfeiçoamento:

Leia atentamente a rubrica colocada no enunciado do Projeto 1 (última página). <br>
Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nas mensagens, mas tendem a melhorar na classificação das mensagens. Ex: stemming, lemmatization, stopwords.
* CONSIDEROU arquivo com três categorias na classificação das variáveis (OBRIGATÓRIO PARA QUARTETOS, sem contar como item avançado)
* CONSTRUIU o cálculo das probabilidades corretamente utilizando bigramas E apresentou referência sobre o método utilizado.
* EXPLICOU porquê não pode usar novas mensagens classificadas pelo próprio classificador como amostra de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto (pelo menos dois cenários diferentes, exceto aqueles já apresentados em sala pelos professores: por exemplo, filtro de spam)
* REFLETE criticamente sobre os resultados obtidos, identificando limitações do modelo e sugerindo possíveis melhorias ou diferentes abordagens com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa).
* DOCUMENTOU bem o código, com explicações claras para cada etapa do processo, incluindo o raciocínio por trás das decisões de modelagem e das transformações de dados.


___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[Natural Language Processing (Part 17)-Laplacian Smoothing](https://medium.com/@Coursesteach/natural-language-processing-part-17-laplacian-smoothing-7d4be71d0ded) **Mais simples**